In [1]:
import pandas as pd
import re
import nltk

In [34]:
df=pd.read_csv("E:\\archive\\train.csv")
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [35]:
def cleanSentence(sentence):
    sentence=re.sub(r'#[A-Za-z0-9]\S+','', sentence)
    sentence=re.sub(r'http\S+', '', sentence)
    sentence=re.sub(r'@[A-Za-z0-9]\S+','',sentence)
    sentence=" ".join(word for word in sentence.split() if len(word)>3)
    return sentence.strip()

In [36]:
df['tweet']=df['tweet'].map(cleanSentence)
df.head

<bound method NDFrame.head of         id  label                                              tweet
0        1      0                                               Test
1        2      0      Finally transparant silicon case Thanks uncle
2        3      0                                   love this! Would
3        4      0                        wired know George made that
4        5      1  What amazing service! Apple won't even talk ab...
...    ...    ...                                                ...
7915  7916      0                                          Live loud
7916  7917      0  would like wish amazing day! Make every minute...
7917  7918      0  Helping lovely year neighbor with iPad this mo...
7918  7919      0           Finally stay connected anytime,anywhere!
7919  7920      0                                 Apple Barcelona!!!

[7920 rows x 3 columns]>

In [37]:
# nltk.download('punkt')

In [38]:
words=[]
for sentences in df['tweet']:
    words.extend(re.split(r'\W+',sentences))
words[:10],len(words)

(['Test',
  'Finally',
  'transparant',
  'silicon',
  'case',
  'Thanks',
  'uncle',
  'love',
  'this',
  'Would'],
 52894)

In [39]:
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
words=set([lemmatizer.lemmatize(word.lower()) for word in words])
len(words)

8038

In [40]:
train_x=[]
words=list(words)
for sentence in df.tweet:
    word=set([word.lower() for word in sentence.split(" ")])
    temp=[]
    for w in words:
        temp.append(1 if w in word else 0)
    train_x.append(temp)
train_x[0].index(1)

5456

In [41]:
X=train_x
Y=df.label

In [42]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [54]:
model=tf.keras.models.Sequential()
model.add(Dense(500,input_shape=(len(words),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

adam=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2)
model.fit(np.array(X_train),np.array(Y_train),epochs=5,batch_size=5)

Epoch 1/5
1268/1268 [==============================] - 31s 24ms/step - loss: 0.4503 - accuracy: 0.8032
Epoch 2/5
1268/1268 [==============================] - 30s 24ms/step - loss: 0.3795 - accuracy: 0.8346
Epoch 3/5
1268/1268 [==============================] - 30s 24ms/step - loss: 0.3393 - accuracy: 0.8456
Epoch 4/5
1268/1268 [==============================] - 30s 24ms/step - loss: 0.2892 - accuracy: 0.8602
Epoch 5/5
1268/1268 [==============================] - 31s 24ms/step - loss: 0.2543 - accuracy: 0.8808


In [80]:
count=0
for i in range(len(X_test)):
    prediction=model.predict(np.array([X_test[i]]))[0][0]
    if prediction>0.5 and np.array(Y_test)[i]==1:
        count+=1
    elif prediction<=0.5 and np.array(Y_test)[i]==0:
        count+=1
        
count/len(X_test)

0.8345959595959596